##### **This notebook sends incomplete jobs again**

In [22]:
import pandas as pd
import numpy as np
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import json
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config
import helper.pandas_helper as pd_hp
import helper.twitter_helper as twitter_hp

##### **Read config files**

In [34]:
config = config.config()
path = config['PATHS']

derived_path = path['derived_path']
all_tweet_data = path['all_tweet_data']
plot_path = path['plot_path']
conversation_path = path['conversation_path']

conversation_ids_5 = path['conversation_ids_5']
conv_ids_5_path = os.path.join(conversation_path, 
                               conversation_ids_5)

##### **Create the temporary files of incomplete conversation ids**

In [74]:
extracted_reply_5 = path['extracted_reply_5']
temp_extracted_json_path = os.path.join(extracted_reply_5, f'*temp.jsonl')

slurm_path = config['SLURM_PATH']
job_log_path = slurm_path['job_log_path']

multiple_job = path['multiple_job']

conv_splited_ids = path['splited_reply_ids']
conv_splited_ids_files = os.path.join(conv_splited_ids, f'*.txt')
temp_conv_splited_ids_files = os.path.join(conv_splited_ids, f'temp.txt')

In [75]:
def check_if_error_terminated(error_path):
    '''
    Checks if the job is terminated due to error
    :param error_path: path where error file is present
    
    :return Boolean
    '''
    error_string = file_hp.read_file(error_path)[-1]
    
    return 'Terminated' in error_string

In [76]:
import imp

imp.reload(twitter_hp)

<module 'helper.twitter_helper' from '/geode2/home/u070/potem/Quartz/project/infoOps-strategy/package/helper/twitter_helper.py'>

In [80]:
def get_remaining_ids(conv_splited_ids_files,
                      extracted_json_path,
                      job_log_path):
    '''
    Gets remaining ids of conversations if the job is terminated
    :param conv_splited_ids_files: spitted id files
    :param extracted_json_path: extracted json files
    :param slurm_script_path: path to slurm script files
    :param job_log_path: path to error files
    '''
    for file in glob.glob(extracted_json_path): 
        print(file)
        parts = (file.split(os.sep)[-1]).split('.')[0]
        error_path = os.path.join(job_log_path, f'{parts}*.err')
        error_file = glob.glob(error_path)
        
        if len(error_file) == 0:
            continue
        
        error_file = error_file[-1]
        if check_if_error_terminated(error_file) == False:
            continue
            
        df = twitter_hp.parse_tweets(file)
        
        if len(df) == 0:
            continue
        
        conv_id_path = os.path.join(conv_splited_ids, parts + '.txt')
        ids = file_hp.read_file(conv_id_path)
        set_ids = set(ids)
        set_df = set(df['conversation_id'].unique())
        
        if len(set_ids) > len(set_df):
            remaining = list(set_ids - set_df)
            filename = f'{parts}_1.txt'
            
            print(filename , ' has remaining ', len(remaining))

            file_hp.write_to_file_row_each_line(conv_splited_ids,
                                            filename,
                                            remaining
                                           )
            
# get_remaining_ids(temp_conv_splited_ids_files,
#                   temp_extracted_json_path,
#                   job_log_path)

##### **Create slurm script for incomplete ids**

In [82]:
def create_jobs(path_conversation,
                python_script,
                path_slurm,
                files
               ):
    '''
    Path of splited conversations
    :param path_conversation: path to conversation ids
    :param path_slurm: path where slurm script to be saved
    '''
    new_path = os.path.join(path_conversation, f'{files}')
    i = 0
    for file in glob.glob(new_path):
        print(file)
        i = i + 1
        parts = file.split(os.sep)
        filename = (parts[-1]).split('.')[0]
        command = f'python {python_script} --file={file}'
        job_name = f'{filename}'
        
        
        script = \
f'''#!/bin/bash -i

#SBATCH --job-name={job_name}
#SBATCH -p general
#SBATCH -o {job_name}_%j.txt
#SBATCH -e {job_name}_%j.err
#SBATCH --mail-type=ALL
#SBATCH --mail-user=potem@iu.edu
#SBATCH --nodes=1
#SBATCH --time=3-00:00:00
#SBATCH --cpus-per-task=2 # Request that ncpus be allocated per process.
#SBATCH --mem=150GB


#Load any modules that your program needs
source /N/u/potem/Quartz/miniconda3/etc/profile.d/conda.sh
conda activate

#cd /N/u/potem/Quartz/sbatch/multiple_jobs/logs
#Run your program
srun {command}
        '''
        new_slurm_path = os.path.join(path_slurm, f'{job_name}.sh')
        
        with open(new_slurm_path, 'w') as f:
            f.write(script)

            
script_path = '/N/u/potem/Quartz/project/infoOps-strategy/script/py_scripts/data_sourcing/get_conversation.py'
slurm_path = path['multiple_job']

# create_jobs(conv_splited_ids, 
#             script_path,
#             slurm_path, 
#             '*_temp_1.txt')

##### **Dispatch the job**

In [83]:
def despatch_job(path, logs_path):
    '''
    Despatch the jobs
    :param path: path where slurm scripts are
    :param logs_path: path where the slurm script to be dispatched
    '''
    all_job_ids = []
    for file in glob.glob(path):
        job_index = file.split('/')[-1].split('_')[1]
        
        if int(job_index) > 60:
            continue
            
        command = f'sbatch {file}'
        
        os.chdir(logs_path)
        
        id = os.system(command)
        
        all_job_ids.append(id)
        
    return all_job_ids

total = 3000

new_path = os.path.join(multiple_job, f'*_temp_1.sh')
all_jobs = despatch_job(new_path, job_log_path)

Submitted batch job 1463113
Submitted batch job 1463114
Submitted batch job 1463115
Submitted batch job 1463116
Submitted batch job 1463117
Submitted batch job 1463118
Submitted batch job 1463119
Submitted batch job 1463120
Submitted batch job 1463121
Submitted batch job 1463122
Submitted batch job 1463123
Submitted batch job 1463124
Submitted batch job 1463125
Submitted batch job 1463126
Submitted batch job 1463127
Submitted batch job 1463128
Submitted batch job 1463129
Submitted batch job 1463130
Submitted batch job 1463131
Submitted batch job 1463132
Submitted batch job 1463133
Submitted batch job 1463134
Submitted batch job 1463135
Submitted batch job 1463136
Submitted batch job 1463137


In [16]:
def cancel_jobs(all_jobs, start_no, job_log_path):
    for i, job in enumerate(all_jobs):
        id = start_no + i

        command = f'scancel -u potem {id}'

        os.chdir(job_log_path)

        os.system(command)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]